In [ ]:
import os
import cv2
from ultralytics import YOLO

# Load the YOLO model
model = YOLO('C:/Users/chira/Desktop/best_best/weights/best.pt')

# Set the input and output folders
input_folder =  'C:/Users/chira/Desktop/best_best/test_new'
output_folder = 'C:/Users/chira/Desktop/best_best/output'

# Ensure output directory exists
os.makedirs(output_folder, exist_ok=True)

# Set confidence threshold
conf_threshold = 0.5  # Confidence threshold for detection

# Process each file in the input folder
for file_name in os.listdir(input_folder):
    file_path = os.path.join(input_folder, file_name)

    # Handle image files
    if file_name.lower().endswith(('.jpg', '.jpeg', '.png', '.bmp', '.tif', '.tiff')):
        img = cv2.imread(file_path)
        if img is None:
            print(f"Could not read image {file_name}. Skipping...")
            continue

        # Perform inference
        results = model(img)

        # Initialize counter for persons detected
        person_count = 0

        # Collect predictions and draw bounding boxes
        for box in results[0].boxes:
            if box.conf[0] >= conf_threshold:
                class_id = int(box.cls[0])
                if class_id == 0:  # Class for 'person'
                    person_count += 1
                    x1, y1, x2, y2 = map(int, box.xyxy[0])
                    cv2.rectangle(img, (x1, y1), (x2, y2), (0, 255, 0), 2)  # Green box
                    cv2.putText(img, f'Person {person_count}', (x1, y1 - 10),
                                cv2.FONT_HERSHEY_SIMPLEX, 0.5, (0, 255, 0), 2)

        # Display the total count on the image
        cv2.putText(img, f'Total People: {person_count}', (10, 30),
                    cv2.FONT_HERSHEY_SIMPLEX, 1, (0, 0, 255), 2)
        
        # Save the processed image
        output_img_path = os.path.join(output_folder, file_name)
        cv2.imwrite(output_img_path, img)
        print(f"{file_name}: {person_count} people detected")

    # Handle video files
    elif file_name.lower().endswith(('.mp4', '.avi', '.mov', '.mkv')):
        cap = cv2.VideoCapture(file_path)
        if not cap.isOpened():
            print(f"Could not read video {file_name}. Skipping...")
            continue

        # Define the video writer
        fourcc = cv2.VideoWriter_fourcc(*'mp4v')
        output_video_path = os.path.join(output_folder, file_name)
        fps = int(cap.get(cv2.CAP_PROP_FPS))
        frame_width = int(cap.get(cv2.CAP_PROP_FRAME_WIDTH))
        frame_height = int(cap.get(cv2.CAP_PROP_FRAME_HEIGHT))
        out = cv2.VideoWriter(output_video_path, fourcc, fps, (frame_width, frame_height))

        while cap.isOpened():
            ret, frame = cap.read()
            if not ret:
                break

            # Perform inference
            results = model(frame)

            # Initialize counter for persons detected
            person_count = 0

            # Collect predictions and draw bounding boxes
            for box in results[0].boxes:
                if box.conf[0] >= conf_threshold:
                    class_id = int(box.cls[0])
                    if class_id == 0:  # Class for 'person'
                        person_count += 1
                        x1, y1, x2, y2 = map(int, box.xyxy[0])
                        cv2.rectangle(frame, (x1, y1), (x2, y2), (0, 255, 0), 2)  # Green box
                        cv2.putText(frame, f'Person {person_count}', (x1, y1 - 10),
                                    cv2.FONT_HERSHEY_SIMPLEX, 0.5, (0, 255, 0), 2)

            # Display the total count on the frame
            cv2.putText(frame, f'Total People: {person_count}', (10, 30),
                        cv2.FONT_HERSHEY_SIMPLEX, 1, (0, 0, 255), 2)

            # Write the frame to the output video
            out.write(frame)

        cap.release()
        out.release()
        print(f"{file_name} processed and saved.")
    else:
        print(f"Unsupported file type for {file_name}. Skipping...")

print("Processing complete.")
